In [1]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()
# OpenAI API key
api_key =os.getenv('OPENAI_API_KEY')
#Delete the api_key variable before pushing to the github.

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory =r'C:\Users\Shaheer\Desktop\Lang-Chain-Projct\Docs_Chroma'
embeddings=OpenAIEmbeddings()
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_directory
)

In [4]:
print(vectordb._collection.count())

209


In [5]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [6]:
small_db =Chroma.from_texts(texts,embedding=embeddings)

In [7]:
question = question = "Tell me about all-white mushrooms with large fruiting bodies"

In [8]:
small_db.similarity_search(question,k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [10]:
small_db.max_marginal_relevance_search(question,k=3,fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [22]:
question1="What did they say about MATLAB ?"
docs_ss = vectordb.similarity_search(question1,k=3)

In [23]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [24]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [25]:
docs_mmr = vectordb.max_marginal_relevance_search(question1,k=3)

In [26]:
docs_mmr[2].page_content[:100]

'over the last 15, 20 years in an electronic format.  \nTurns out that most of you probably use learni'

In [27]:
question = "what did they say about regression in the third lecture?"

In [31]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":r"C:\Users\Shaheer\Desktop\Lang-Chain-Projct\Docs\MachineLearning-Lecture03.pdf"}
)

In [32]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'C:\\Users\\Shaheer\\Desktop\\Lang-Chain-Projct\\Docs\\MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': 'C:\\Users\\Shaheer\\Desktop\\Lang-Chain-Projct\\Docs\\MachineLearning-Lecture03.pdf'}
{'page': 4, 'source': 'C:\\Users\\Shaheer\\Desktop\\Lang-Chain-Projct\\Docs\\MachineLearning-Lecture03.pdf'}


### TFIDF & SVM

In [46]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [47]:
# Load PDF
loader = PyPDFLoader(r"C:\Users\Shaheer\Desktop\Lang-Chain-Projct\Docs\MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [48]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embeddings)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [49]:
question = "What are major topics for this class?"
docs_svm=svm_retriever.get_relevant_documents(question)
docs_svm[0]

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Document(page_content="let me just check what questions you have righ t now. So if there are no questions, I'll just \nclose with two reminders, which are after class today or as you start to talk with other \npeople in this class, I just encourage you again to start to form project partners, to try to \nfind project partners to do your project with. And also, this is a good time to start forming \nstudy groups, so either talk to your friends  or post in the newsgroup, but we just \nencourage you to try to star t to do both of those today, okay? Form study groups, and try \nto find two other project partners.  \nSo thank you. I'm looking forward to teaching this class, and I'll see you in a couple of \ndays.   [End of Audio]  \nDuration: 69 minutes")

In [50]:
question = "what did they say about matlab?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste